# Anomaly Detection with Real-world Data
In this notebook, we conduct an experiment where we train anomaly detection algorithms on snythetic data and test them on real-world data.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/berni-lehner/structural_health_monitoring/blob/main/notebooks/mixed_anomaly_AB.ipynb)

In [1]:
random_state = None

## Prerequisites

In [2]:
import sys
import os
from pathlib import Path
import pandas as pd


# initialize environment, paths, etc...
IN_COLAB = 'google.colab' in sys.modules

if(IN_COLAB):
    # uncomment in case you want to start from scratch in Colab
    !rm -rf /content/structural_health_monitoring  
    !git clone https://github.com/berni-lehner/structural_health_monitoring.git
    !pip install -r /content/structural_health_monitoring/requirements.txt
    
    # this is Colab specific, and only a manual "Run all" will cause the
    # installation to be finished
    if(pd.__version__ != '1.4.2'):
      print('Stopping RUNTIME! Please run again to finish installation.')
      os.kill(os.getpid(), 9)

    DATA_PATH = Path(r"/content/structural_health_monitoring/data")
    sys.path.insert(0, "/content/structural_health_monitoring/src")
else:
    DATA_PATH = Path(r"../data/")
    sys.path.insert(0,"../src")
    
    
sys.path.insert(0, DATA_PATH)
SYN_DATA_PATH = Path(DATA_PATH, "synthetic")
REAL_DATA_PATH = Path(DATA_PATH, "real_world")
RESULT_PATH = Path(DATA_PATH, "results")
if not os.path.isdir(RESULT_PATH):
    os.makedirs(RESULT_PATH)

try:
    from zippeekiyay import namelist
except ImportError or ModuleNotFoundError:
    print('installing zippee-ki-yay...')
    !pip install git+https://github.com/berni-lehner/zippee-ki-yay.git

    from zippeekiyay import namelist

CALIBRATION_FILE = Path(REAL_DATA_PATH, "calibration.pkl")

In [4]:
import time
import glob
import sklearn
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.pipeline import Pipeline
import numpy as np
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

#...and download raw data if not already downloaded
from data_utils import init_data
init_data(syn_data_path=SYN_DATA_PATH, real_data_path=REAL_DATA_PATH)

from plot_utils import (
    init_plotting,
    plot_metrics,
    plot_cv_indices,
    plot_embedding_targets,
    init_plotting,
    plot_classwise_dist,
    plot_classwise_kde,
)

# configure fonts, etc...
init_plotting()

from scoring_utils import get_anomaly_scoring
from cv_utils import (
    AnomalyShuffleSplit,
    RepeatedAnomalyShuffleSplit,
    CreateAnomalyData,
    dump_cv)
from bench_utils import (
    classification_benchmark,
    repeat_experiment,
    aabb_classification_benchmark,
    extract_metrics)
from data_utils import (load_processed_data,
                        load_syn_reg_data,
                        FEATURE_LIST)

time passed: 5.83 s
downloading synthetic data successful: True
time passed: 1.47 s
downloading real world data successful: True


In [5]:
result_file = "mixed_anomaly_AB.pkl"
result_file = Path(RESULT_PATH, result_file)

# Data Loading

## Synthetic Data

In [6]:
Xsyn, ysyn = load_syn_reg_data(data_path=SYN_DATA_PATH,
                               min_radius = 1.9,
                               cache=True)

In [7]:
Xsyn.shape

(28302, 31)

In [8]:
ctr = Counter(ysyn)
ctr

Counter({0.0: 990,
         2.2: 979,
         2.5: 979,
         2.8: 978,
         3.1: 986,
         3.4: 972,
         3.8: 973,
         4.1: 976,
         4.4: 979,
         4.7: 976,
         5.0: 977,
         6.7: 981,
         8.3: 972,
         10.0: 973,
         12.0: 969,
         14.0: 978,
         16.0: 980,
         18.0: 976,
         20.0: 974,
         22.0: 975,
         24.0: 980,
         26.0: 973,
         28.0: 983,
         30.0: 977,
         32.0: 967,
         34.0: 971,
         36.0: 976,
         38.0: 964,
         40.0: 968})

## Real World Data

In [9]:
Xreal, yreal = load_syn_reg_data(data_path=REAL_DATA_PATH,
                                 min_radius = 1.,
                                 synthetic=False,
                                 calibration_file=CALIBRATION_FILE)

In [10]:
Xreal.shape

(225, 31)

In [11]:
ctr = Counter(yreal)
ctr

Counter({0.0: 9,
         1.5: 9,
         10.0: 9,
         12.0: 9,
         14.0: 9,
         16.0: 9,
         17.0: 9,
         19.0: 9,
         2.5: 9,
         21.0: 9,
         23.0: 9,
         25.0: 9,
         27.0: 9,
         29.0: 9,
         3.5: 9,
         31.0: 9,
         33.0: 9,
         35.0: 9,
         37.0: 9,
         39.0: 9,
         4.5: 9,
         40.0: 9,
         5.5: 9,
         7.0: 9,
         8.0: 9})

## Combine both datasets and turn target variable into anomaly labels

In [12]:
# combine to full data set
X = np.concatenate([Xsyn, Xreal], axis=0)
y = np.concatenate([ysyn, yreal], axis=0)

y_anomaly = y.copy()
y_anomaly[y != 0] = -1
y_anomaly[y == 0] = 1


ctr = Counter(y_anomaly)
ctr

Counter({1.0: 999, -1.0: 27528})

## sanity check label distribution of real-world data

In [13]:
ctr = Counter(y_anomaly[len(Xsyn):])
ctr

Counter({1.0: 9, -1.0: 216})

In [14]:
n_repeats = 1

cv = RepeatedAnomalyShuffleSplit(Xpos=Xsyn, Xneg=Xreal, n_splits=128, test_size=0.05, n_repeats=n_repeats,
                                 random_state=random_state, unseen_only=True)

In [15]:
scoring = get_anomaly_scoring()

In [16]:
X.shape

(28527, 31)

# AB Testing

In [17]:
estimator = IsolationForest(contamination=0.5, n_estimators=100)
pipeline = Pipeline([('est', estimator)])

if_pipelines = [("IsolationForest", pipeline)]

In [18]:
estimator = OneClassSVM(gamma='scale', nu=0.5)
pipeline = Pipeline([('est', estimator)])

svm_pipelines = [("OneClassSVM", pipeline)]

In [19]:
estimator = EllipticEnvelope(contamination=0.4)
pipeline = Pipeline([('est', estimator)])

mcd_pipelines = [("EllipticEnvelope", pipeline)]

In [20]:
svm_pipelines

[('OneClassSVM', Pipeline(steps=[('est', OneClassSVM())]))]

In [21]:
if_pipelines

[('IsolationForest',
  Pipeline(steps=[('est', IsolationForest(contamination=0.5))]))]

In [22]:
mcd_pipelines

[('EllipticEnvelope',
  Pipeline(steps=[('est', EllipticEnvelope(contamination=0.4))]))]

In [ ]:
results = aabb_classification_benchmark(X=X, y=y_anomaly,
                                   models=[if_pipelines, svm_pipelines, mcd_pipelines],
                                   cv=cv,
                                   scoring=scoring,
                                   random_state=random_state)

## Store results for later

In [ ]:
results.to_pickle(result_file)

## Plot some results for overview

In [ ]:
for key in results.columns[2:4]:
    fig = plot_metrics(extract_metrics(results, [key]))
    plt.show()